In [11]:
import os
import pandas as pd, numpy as np
from urbansim.models import transition
import orca
from urbansim.utils import misc
import pandas as pd

In [2]:
# import models, utils
# from urbansim.utils import misc, networks

#### Import the simulation framework

In [ ]:
#import urbansim.sim.simulation as sim

#### Connect to data store, register it as an injectable

In [12]:
semcog_hdfstore = pd.HDFStore("data/run4032_school_v2_baseyear.h5", mode="r")
orca.add_injectable("store", semcog_hdfstore)

In [13]:
semcog_hdfstore

<class 'pandas.io.pytables.HDFStore'>
File path: data/run4032_school_v2_baseyear.h5

#### Register data tables that will be used in the simulation

In [14]:
@orca.table('jobs') 
def jobs(store): 
    df = store['jobs']
    return df

@orca.table('buildings') 
def jobs(store): 
    df = store['buildings']
    return df

@orca.table('parcels') 
def jobs(store): 
    df = store['parcels']
    return df

@orca.table('annual_employment_control_totals')
def annual_employment_control_totals(store):
    df = store['annual_employment_control_totals']
    return df

In [15]:
cols = ['walk_nearest_urgentcare',
     'zones_logsum_job_low_income',
     'bldg_empratio_17',
     'residential_units',
     'year_built',
     'b_total_jobs',
     'nodes_walk_lowinc_hhs',
     'nodes_walk_ave_lot_sqft',
     'building_sqft',
     'zones_transit_jobs_50min']
orca.get_table('buildings').to_frame(cols)

,residential_units,year_built
building_id,,
1184091,1,1900
1183390,0,0
1183392,0,0
1183409,0,0
1636985,0,0
1183492,8,1966
1183506,16,1966
1183608,8,1966
1183357,8,0


In [6]:
buildings = orca.get_table('buildings').to_frame()

In [7]:
buildings.duplicated().any()

True

In [9]:
len(buildings)

1660662

In [12]:
buildings.columns

Index([u'residential_units', u'year_built', u'b_city_id', u'parcel_id',
       u'land_area', u'improvement_value', u'stories', u'building_type_id',
       u'non_residential_sqft', u'b_zone_id', u'sqft_per_unit'],
      dtype='object')

#### Register your models

In [ ]:
@orca.step('jobs_transition')
def jobs_transition(jobs, annual_employment_control_totals,iter_var): 
    ct_emp = annual_employment_control_totals.to_frame()
    ct_emp = ct_emp.reset_index().set_index('year')
    tran = transition.TabularTotalsTransition(ct_emp, 'total_number_of_jobs')
    model = transition.TransitionModel(tran)
    j = jobs.to_frame(jobs.local_columns)
    #print "j"
    new, added_jobs_idx, new_linked = model.transition(j, iter_var)
    new.loc[added_jobs_idx, "building_id"] = -1
    orca.add_table("jobs", new)
    
@orca.step('print_year')
def print_year(iter_var):
    print '*** the year is {} ***'.format(iter_var)

#### A demonstration of running the above models

In [ ]:
#jobs = orca.get_table('jobs')
#print jobs.to_frame().describe()

In [ ]:
jobs.to_frame().head()

In [ ]:
orca.run(["print_year","jobs_transition"], iter_vars=[2010], data_out='runs/test_run.h5', out_interval=1)

In [ ]:
orca.get_table('jobs').to_frame().describe()

In [ ]:
print orca._TABLES
print orca._MODELS
print orca._INJECTABLES

##show add_injectable and how it updates the _INJECTABLES list, then show the dictionaries up at top of sim (empty dicts)

In [18]:
orca.run(["refiner",
          'build_networks'])

Running step 'refiner'
Time to execute step 'refiner': 1.77 s
Running step 'build_networks'
Time to execute step 'build_networks': 179.35 s
Total time to execute iteration 1 with iteration value None: 181.14 s


In [19]:
b = orca.get_table('buildings').to_frame(['large_area_id'])
j = orca.get_table('jobs').to_frame()
h = orca.get_table('households').to_frame()
idx_invalid_building_id = np.in1d(j.building_id, b.index.values) == False

building_vars = set(orca.get_table('buildings').columns)

# utils._convert_network_types("networks_walk.yaml")
# utils._convert_network_types("networks_drv.yaml")

nodes = networks.from_yaml(orca.get_injectable('net_walk'), "networks_walk.yaml")
# print nodes.describe()
# print pd.Series(nodes.index).describe()
orca.add_table("nodes_walk", nodes)

variables\variables_demographic.py:81: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(households.income)


Computing accessibility variables
Computing log_sum_residential_units
Removed 8 rows because they contain missing values
Computing log_sum_nonresidential_units
Removed 8 rows because they contain missing values
Computing ave_lot_sqft
Removed 8 rows because they contain missing values


ValueError: Length of passed values is 0, index implies 270914

In [9]:
node_col = 'nodeid_walk'
vname = 'persons'
df = orca.get_table('households').to_frame([node_col, vname])
#   - name: ave_lot_sqft
#     dataframe: buildings
#     varname: land_area
#     radius: 500
#     apply: np.log1p
#     decay: flat
#     aggregation: average
net = orca.get_injectable('net_walk')

KeyError: 'table not found: households'

In [10]:
orca.get_table('buildings').to_frame(['land_area'])

,land_area
building_id,
1184091,940
1183390,1760
1183392,1760
1183409,24922
1636985,315358
1183492,4388
1183506,12942
1183608,4470
1183357,3542


In [24]:
net.aggregate()

<pandana.network.Network instance at 0x00000002D5F99088>

In [25]:
net.set(df[node_col], variable=df[vname] if vname else None)
# aggregate it

In [34]:
radius = 1500.00
agg = 'ave'
decay= 'flat'
net.aggregate(radius, type=agg, decay=decay)

0         3.330882
1         2.627358
2         3.130435
3         0.000000
4         2.697183
5         2.395349
6         2.615385
7         3.100000
8         2.434783
9         0.000000
10        3.196078
11        2.574194
12        0.000000
13        2.416667
14        2.685393
15        2.583333
16        0.000000
17        2.537931
18        2.188406
19        2.473684
20        0.000000
21        0.000000
22        2.754266
23        0.000000
24        3.166667
25        2.680000
26        0.000000
27        3.000000
28        3.384615
29        3.107143
            ...   
270884    2.760000
270885    2.474801
270886    2.041667
270887    0.000000
270888    2.413462
270889    2.727273
270890    2.919149
270891    2.433447
270892    2.538462
270893    2.444444
270894    2.333333
270895    1.659574
270896    2.254717
270897    2.812398
270898    2.487805
270899    2.761905
270900    2.133333
270901    1.984536
270902    0.000000
270903    2.171429
270904    2.372587
270905    1.

In [ ]:
# Disaggregate nodal variables to building.
for var in orca.get_table('nodes_walk').columns:
    if var not in building_vars:
        variables.make_disagg_var('nodes_walk', 'buildings', var, 'nodeid_walk')

In [ ]:
def make_disagg_var(from_geog_name, to_geog_name, var_to_disaggregate, from_geog_id_name, name_based_on_geography=True):
    """
    Generator function for disaggregating variables. Registers with orca.
    """
    if name_based_on_geography:
        var_name = from_geog_name + '_' + var_to_disaggregate
    else:
        var_name = var_to_disaggregate

    @orca.column(to_geog_name, var_name, cache=True, cache_scope='iteration')
    def func():
        print 'Disaggregating {} to {} from {}'.format(var_to_disaggregate, to_geog_name, from_geog_name)

        from_geog = orca.get_table(from_geog_name)
        to_geog = orca.get_table(to_geog_name)
        return misc.reindex(from_geog[var_to_disaggregate], to_geog[from_geog_id_name]).fillna(0)

    return func